# Building Fast Queries on a CSV
## Reading the Inventory

In [1]:
import csv

with open('laptops.csv') as f:
    reader = csv.reader(f)
    rows = list(reader)
header = rows[0]
rows = rows[1:]
    
print(header)
print('\n')
for i in range(5):
    print(rows[i])

['laptop_ID', 'Company', 'Product', 'TypeName', 'Inches', 'ScreenResolution', 'Cpu', 'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight', 'Price_euros']


['1', 'Apple', 'MacBook Pro', 'Ultrabook', '13.3', 'IPS Panel Retina Display 2560x1600', 'Intel Core i5 2.3GHz', '8GB', '128GB SSD', 'Intel Iris Plus Graphics 640', 'macOS', '1.37kg', '1339.69']
['2', 'Apple', 'Macbook Air', 'Ultrabook', '13.3', '1440x900', 'Intel Core i5 1.8GHz', '8GB', '128GB Flash Storage', 'Intel HD Graphics 6000', 'macOS', '1.34kg', '898.94']
['3', 'HP', '250 G6', 'Notebook', '15.6', 'Full HD 1920x1080', 'Intel Core i5 7200U 2.5GHz', '8GB', '256GB SSD', 'Intel HD Graphics 620', 'No OS', '1.86kg', '575.00']
['4', 'Apple', 'MacBook Pro', 'Ultrabook', '15.4', 'IPS Panel Retina Display 2880x1800', 'Intel Core i7 2.7GHz', '16GB', '512GB SSD', 'AMD Radeon Pro 455', 'macOS', '1.83kg', '2537.45']
['5', 'Apple', 'MacBook Pro', 'Ultrabook', '13.3', 'IPS Panel Retina Display 2560x1600', 'Intel Core i5 3.1GHz', '8GB', '256GB SSD', 'I

## Inventory Class

In [2]:
class Inventory():                    
    
    def __init__(self, csv_filename): 
        with open(csv_filename) as f: 
            reader = csv.reader(f)
            rows = list(reader)
        self.header = rows[0]         
        self.rows = rows[1:]
        for row in self.rows:
            row[-1] = int(float(row[-1]))

inventory = Inventory('laptops.csv')  
print(inventory.header)            
print(len(inventory.rows))            

['laptop_ID', 'Company', 'Product', 'TypeName', 'Inches', 'ScreenResolution', 'Cpu', 'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight', 'Price_euros']
1303


## Finding a Laptop From the Id

In [3]:
class Inventory():                    
    
    def __init__(self, csv_filename): 
        with open(csv_filename) as f: 
            reader = csv.reader(f)
            rows = list(reader)
        self.header = rows[0]         
        self.rows = rows[1:]
        for row in self.rows:
            row[-1] = int(float(row[-1]))
            
    def get_laptop_from_id(self, laptop_id):   
        for row in self.rows:                  
            if row[0] == laptop_id:
                return row
        return None

In [4]:
inventory = Inventory('laptops.csv')         
print(inventory.get_laptop_from_id('1300')) 
print(inventory.get_laptop_from_id('3563'))

['1300', 'HP', 'Stream 11-Y000na', 'Netbook', '11.6', '1366x768', 'Intel Celeron Dual Core N3060 1.6GHz', '2GB', '32GB Flash Storage', 'Intel HD Graphics 400', 'Windows 10', '1.17kg', 209]
None


## Improving Id Lookups

In [5]:
class Inventory():                    
    
    def __init__(self, csv_filename): 
        with open(csv_filename) as f: 
            reader = csv.reader(f)
            rows = list(reader)
        self.header = rows[0]         
        self.rows = rows[1:]
        for row in self.rows:
            row[-1] = int(float(row[-1]))
        self.id_to_row = {}                      
        for row in self.rows:                       
            self.id_to_row[row[0]] = row 
            
    def get_laptop_from_id(self, laptop_id):   
        for row in self.rows:                  
            if row[0] == laptop_id:
                return row
        return None  
    
    def get_laptop_from_id_fast(self, laptop_id):  
        if laptop_id in self.id_to_row:             
            return self.id_to_row[laptop_id]
        return None

### Test the code:

In [6]:
inventory = Inventory('laptops.csv')           
print(inventory.get_laptop_from_id('1300'))
print(inventory.get_laptop_from_id('3563'))

['1300', 'HP', 'Stream 11-Y000na', 'Netbook', '11.6', '1366x768', 'Intel Celeron Dual Core N3060 1.6GHz', '2GB', '32GB Flash Storage', 'Intel HD Graphics 400', 'Windows 10', '1.17kg', 209]
None


## Comparing Performance

In [7]:
import time                                                         
import random                                                       

ids = [str(random.randint(1, 5000)) for _ in range(10000)] # step 3

inventory = Inventory('laptops.csv')                                

total_time_no_dict = 0                                              
for identifier in ids:                                              
    start = time.time()                                             
    inventory.get_laptop_from_id(identifier)                        
    end = time.time()                                               
    total_time_no_dict += end - start                              
    
total_time_dict = 0                                                 
for identifier in ids:                                              
    start = time.time()                                            
    inventory.get_laptop_from_id_fast(identifier)                   
    end = time.time()                                               
    total_time_dict += end - start                                  
    
print(total_time_no_dict)                                           
print(total_time_dict)

0.7950553894042969
0.001999378204345703


### Analysis

We got:

time1 = 0.8430628776550293

time2 = 0.00400233268737793

We can see a significant improve in performance. If we divide time1 by time2:

In [8]:
time1 = 0.8430628776550293

time2 = 0.00400233268737793

print(time1/time2)

210.6428784178233


We see that the new method is about 211 times faster for this input size.

## Two Laptop Promotion

In [9]:
class Inventory():                    
    
    def __init__(self, csv_filename): 
        with open(csv_filename) as f: 
            reader = csv.reader(f)
            rows = list(reader)
        self.header = rows[0]         
        self.rows = rows[1:]
        for row in self.rows:
            row[-1] = int(float(row[-1]))
        self.id_to_row = {}                      
        for row in self.rows:                       
            self.id_to_row[row[0]] = row 
            
    def get_laptop_from_id(self, laptop_id):   
        for row in self.rows:                  
            if row[0] == laptop_id:
                return row
        return None  
    
    def get_laptop_from_id_fast(self, laptop_id):  
        if laptop_id in self.id_to_row:             
            return self.id_to_row[laptop_id]
        return None
    
    def check_promotion_dollars(self, dollars):    
        for row in self.rows:                      
            if row[-1] == dollars:
                return True
        for row1 in self.rows:                     
            for row2 in self.rows:
                if row1[-1] + row2[-1] == dollars:
                    return True
        return False  

In [10]:
inventory = Inventory('laptops.csv')               
print(inventory.check_promotion_dollars(1000))     
print(inventory.check_promotion_dollars(442))

True
False


## Optimizing Laptop Promotion

In [11]:
class Inventory():                    
    
    def __init__(self, csv_filename): 
        with open(csv_filename) as f: 
            reader = csv.reader(f)
            rows = list(reader)
        self.header = rows[0]         
        self.rows = rows[1:]
        for row in self.rows:
            row[-1] = int(float(row[-1]))
        self.id_to_row = {}                      
        for row in self.rows:                       
            self.id_to_row[row[0]] = row
        self.prices = set()                          
        for row in self.rows:                        
            self.prices.add(row[-1])
            
    def get_laptop_from_id(self, laptop_id):   
        for row in self.rows:                  
            if row[0] == laptop_id:
                return row
        return None  
    
    def get_laptop_from_id_fast(self, laptop_id):  
        if laptop_id in self.id_to_row:             
            return self.id_to_row[laptop_id]
        return None
    
    def check_promotion_dollars(self, dollars):    
        for row in self.rows:                      
            if row[-1] == dollars:
                return True
        for row1 in self.rows:                     
            for row2 in self.rows:
                if row1[-1] + row2[-1] == dollars:
                    return True
        return False 
    
    def check_promotion_dollars_fast(self, dollars): 
        if dollars in self.prices:                   
            return True
        for price in self.prices:                    
            if dollars - price in self.prices:
                return True
        return False

### Test the code:

In [12]:
inventory = Inventory('laptops.csv')                 
print(inventory.check_promotion_dollars_fast(1000))  
print(inventory.check_promotion_dollars_fast(442))

True
False


## Comparing Promotion Functions

In [13]:
prices = [random.randint(100, 5000) for _ in range(100)]

inventory = Inventory('laptops.csv')                     

total_time_no_set = 0                                    
for price in prices:                                     
    start = time.time()                                 
    inventory.check_promotion_dollars(price)             
    end = time.time()                                    
    total_time_no_set += end - start                     
    
total_time_set = 0                                     
for price in prices:                                     
    start = time.time()                               
    inventory.check_promotion_dollars_fast(price)       
    end = time.time()                                  
    total_time_set += end - start                        
    
print(total_time_no_set)                                 
print(total_time_set)

0.9840719699859619
0.0


### Analysis

We got:

time1 = 1.396103858947754

time2 = 0.001001119613647461

We can see a significant improve in performance. If we divide time1 by time2:

In [14]:
time1 = 1.396103858947754

time2 = 0.001001119613647461

print(time1/time2)

1394.5425101214576


We see that the new method is about 1395 times faster for this input size.

## Finding Laptops Within a Budget

In [15]:
def row_price(row):
    return row[-1]

class Inventory():                    
    
    def __init__(self, csv_filename): 
        with open(csv_filename) as f: 
            reader = csv.reader(f)
            rows = list(reader)
        self.header = rows[0]         
        self.rows = rows[1:]
        for row in self.rows:
            row[-1] = int(float(row[-1]))
        self.id_to_row = {}                      
        for row in self.rows:                       
            self.id_to_row[row[0]] = row
        self.prices = set()                          
        for row in self.rows:                        
            self.prices.add(row[-1])
        self.rows_by_price = sorted(self.rows, key=row_price)
            
    def get_laptop_from_id(self, laptop_id):   
        for row in self.rows:                  
            if row[0] == laptop_id:
                return row
        return None  
    
    def get_laptop_from_id_fast(self, laptop_id):  
        if laptop_id in self.id_to_row:             
            return self.id_to_row[laptop_id]
        return None
    
    def check_promotion_dollars(self, dollars):    
        for row in self.rows:                      
            if row[-1] == dollars:
                return True
        for row1 in self.rows:                     
            for row2 in self.rows:
                if row1[-1] + row2[-1] == dollars:
                    return True
        return False 
    
    def check_promotion_dollars_fast(self, dollars): 
        if dollars in self.prices:                   
            return True
        for price in self.prices:                    
            if dollars - price in self.prices:
                return True
        return False
    
    def find_first_laptop_more_expensive(self, target_price): 
        range_start = 0                                   
        range_end = len(self.rows_by_price) - 1                   
        while range_start < range_end:
            range_middle = (range_end + range_start) // 2  
            price = self.rows_by_price[range_middle][-1]
            if price > target_price:
                range_end = range_middle
            else:
                range_start = range_middle + 1
        if self.rows_by_price[range_start][-1] <= target_price:                  
            return -1                                   
        return range_start

inventory = Inventory('laptops.csv')                                 
print(inventory.find_first_laptop_more_expensive(1000))  
print(inventory.find_first_laptop_more_expensive(10000)) 

683
-1
